In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint
import matplotlib.dates as plt_date
from scipy.interpolate import interp1d
from scipy.interpolate import make_interp_spline
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import Perceptron
import PIL


from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, \
    f1_score, confusion_matrix

from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from xml.sax.saxutils import escape
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

from datetime import datetime as dt
import tensorflow

2023-04-18 10:37:49.788787: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-18 10:37:49.808774: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 10:37:50.191105: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print(tensorflow)

<module 'tensorflow' from '/home/yuno/.conda/envs/tf/lib/python3.9/site-packages/tensorflow/__init__.py'>


In [3]:
candidates_df = pd.read_csv("/home/yuno/Getpro/labtech/full_candidates_title.csv")


candidates_df

,title,title.1,?column?
0,Consultant SIRH,Consultant senior SIRH,"[{""end"": null, ""start"": ""2020-06-01"", ""title"":..."
1,Talent Acquisition Manager,NaN,"[{""end"": null, ""start"": ""2020-05-01"", ""title"":..."
2,Ingénieur Système Embarqué,Architecte sous-systèmes,"[{""end"": ""2022-10-31"", ""start"": ""2021-11-29"", ..."
3,Responsable Partenariats Stratégiques,NaN,"[{""end"": null, ""start"": ""2018-01-01"", ""title"":..."
4,Data Scientist,Data scientist,"[{""end"": null, ""start"": ""2020-03-30"", ""title"":..."
...,...,...,...
39564,Talent Acquisition Manager,Talent Acquisition Manager chez CANAL+ INTERNA...,"[{""end"": null, ""start"": ""2021-06-28"", ""title"":..."
39565,Développeur Lead,Lead Developer - BNP Paribas,"[{""end"": null, ""start"": ""2020-04-27"", ""title"":..."
39566,Technicien Support,Technicien assistance applications chez Groupe...,"[{""end"": null, ""start"": ""2017-05-29"", ""title"":..."
39567,Conducteur de travaux,Conducteur de travaux cadre chez Bouygues Ener...,"[{""end"": null, ""start"": ""2020-08-31"", ""title"":..."


In [4]:
candidates_df["?column?"][0]

'[{"end": null, "start": "2020-06-01", "title": "Consultant senior SIRH", "company": "Sopra HR Software", "current": true, "summary": "Gestion et mise en œuvre de projets (planning, préconisations, développement, tests, recette, mise en production, etc.).\xa0\\nRéalisation des opérations de développement, et/ou maintenance, et/ou support de composants applicatifs ou techniques de systèmes d’information, dans le respect des consignes, processus, normes qualité et sécurité, et selon les charges et planning prévus.\xa0\\nPrise en charge de projets (techniques, fonctionnels, d’intégration et de maintenance).\xa0\\nAdaptation des préconisations aux problématiques du client. Accompagnement sur la mise en œuvre.\xa0\\nProcédures établies sous Script UNIX et langage SQL sur la base Oracle.\xa0\\nInstallations des nouvelles versions des logiciels (préconisations, analyses d\'impacts sur le spécifique client, etc.).", "location": "Ville de Paris, Île-de-France, France"}, {"end": "2020-06-01", "s

In [5]:
max_len = 0


In [6]:
import json
MAX_CONTENT_LENGHT = 0

def xp_symmary(xp):
  global MAX_CONTENT_LENGHT
  xp = json.loads(xp)
  res = "\n".join([
    row["title"] + (": " + row["summary"] )if row["summary"] else "" for row in xp if row["title"]
    ])
  if len(res) > MAX_CONTENT_LENGHT:
    MAX_CONTENT_LENGHT = len(res)
  return res

candidates_df["Formated summaries"] = candidates_df["?column?"].apply(xp_symmary)


print(MAX_CONTENT_LENGHT)
candidates_df

18287


,title,title.1,?column?,Formated summaries
0,Consultant SIRH,Consultant senior SIRH,"[{""end"": null, ""start"": ""2020-06-01"", ""title"":...",Consultant senior SIRH: Gestion et mise en œuv...
1,Talent Acquisition Manager,NaN,"[{""end"": null, ""start"": ""2020-05-01"", ""title"":...",Talent manager: En charge du développement de ...
2,Ingénieur Système Embarqué,Architecte sous-systèmes,"[{""end"": ""2022-10-31"", ""start"": ""2021-11-29"", ...",Ingenieur IVV Système: Projet : systèmes avion...
3,Responsable Partenariats Stratégiques,NaN,"[{""end"": null, ""start"": ""2018-01-01"", ""title"":...",Responsable partenariats & projets stratégique...
4,Data Scientist,Data scientist,"[{""end"": null, ""start"": ""2020-03-30"", ""title"":...",Enseignant vacataire - Introduction to chatbot...
...,...,...,...,...
39564,Talent Acquisition Manager,Talent Acquisition Manager chez CANAL+ INTERNA...,"[{""end"": null, ""start"": ""2021-06-28"", ""title"":...",\n\nResearch & Executive Assistant: Executive ...
39565,Développeur Lead,Lead Developer - BNP Paribas,"[{""end"": null, ""start"": ""2020-04-27"", ""title"":...",\n\n\n\n\n
39566,Technicien Support,Technicien assistance applications chez Groupe...,"[{""end"": null, ""start"": ""2017-05-29"", ""title"":...",\nTECHNICIEN SUPPORT IT: Masterisation pour le...
39567,Conducteur de travaux,Conducteur de travaux cadre chez Bouygues Ener...,"[{""end"": null, ""start"": ""2020-08-31"", ""title"":...",\n\n


In [7]:
candidates_df.size

158276

In [8]:
candidates_df = candidates_df[candidates_df["Formated summaries"].apply(lambda x: x != "")]
candidates_df.size

157372

In [9]:
from sklearn import preprocessing
from keras.utils import to_categorical
import pickle

X = candidates_df["Formated summaries"]
Y = candidates_df["title"]
label_encoder = preprocessing.LabelEncoder()

label_encoder.fit(Y)

filehandler = open("label_encoder.pkl","wb")
pickle.dump(label_encoder, filehandler, protocol=pickle.HIGHEST_PROTOCOL)

Y = label_encoder.transform(Y)
Y = to_categorical(Y)

In [10]:
Y.shape

(39343, 293)

In [11]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
# x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

In [12]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import pickle


tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)


filehandler = open("tokenizer.pkl","wb")
pickle.dump(tokenizer, filehandler, protocol=pickle.HIGHEST_PROTOCOL)

x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = MAX_CONTENT_LENGHT

x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
x_test = pad_sequences(x_test, padding='post', maxlen=maxlen)



In [13]:
from keras.models import Sequential
from keras import layers

embedding_dim = 100
def create_model(learning_rate=0.001, dropout=0.4):
    model = Sequential()
    model.add(layers.Embedding(input_dim=vocab_size, 
                               output_dim=embedding_dim, 
                               input_length=maxlen))
    model.add(layers.GlobalMaxPool1D())
    model.add(layers.Dropout(dropout))

    model.add(layers.Dense(Y.shape[1], activation='softmax'))
    model.compile(optimizer='adam',
                  loss="categorical_crossentropy",
                  metrics=['accuracy'])
    return model

model = create_model()


2023-04-18 10:37:59.887038: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-18 10:37:59.900553: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-18 10:37:59.900678: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

weight_path = '{}_title_my_model.best.hdf5'.format('xray_class')

#'weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only= True)
early = EarlyStopping(monitor= 'val_loss', mode= 'min', patience= 100)
callbacks_list = [checkpoint, early]

In [15]:
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [16]:
cnn_model = model.fit(
    x=x_train,
    y=y_train,
    batch_size=32,
    validation_data=(x_test, y_test),
    epochs=10,
    callbacks= callbacks_list
)

Epoch 1/10


2023-04-18 10:38:11.227602: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 223.23MiB (rounded to 234073600)requested by op sequential/embedding/embedding_lookup
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-04-18 10:38:11.227625: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-04-18 10:38:11.227631: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 25, Chunks in use: 25. 6.2KiB allocated for chunks. 6.2KiB in use in bin. 380B client-requested in use in bin.
2023-04-18 10:38:11.227635: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2023-04-18 10:38:11.227640: I tensorflow/tsl

ResourceExhaustedError: Graph execution error:

Detected at node 'sequential/embedding/embedding_lookup' defined at (most recent call last):
    File "/home/yuno/.conda/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/yuno/.conda/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/yuno/.conda/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/yuno/.conda/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/yuno/.conda/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/home/yuno/.conda/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/home/yuno/.conda/en

In [ ]:
y_test.shape

(7903, 293)

In [ ]:
predictions = model.predict(x_test)

print(x_test.shape)
texts = tokenizer.sequences_to_texts(x_test)
index = 0
for p, result in zip(predictions, y_test):
    prediction = np.argmax(p)
    res = np.argmax(result) 

    if res != prediction:
        titles = texts[index]
        category = label_encoder.inverse_transform([prediction])[0]
        res = label_encoder.inverse_transform([res])[0]
        taux = np.max(p)

        print(f"{taux}: {titles} -> predicted: {category} | res: {res}")

    index += 1

247/247 [==============================] - 0s 687us/step
(7903, 1009)
0.5483508110046387: principal software engineer senior research engineer machine learning senior research engineer machine learning senior research engineer machine learning founder lead software developer -> predicted: Machine Learning Engineer | res: Développeur Lead
0.30599337816238403: technical lead full stack developer teacher freelance teacher freelance web developer web developer digital cinema engineer -> predicted: Lead Tech Full stack | res: Développeur Full stack
0.37955334782600403: account manager ingénieur d'affaires grands comptes large accounts sales executive senior business development manager founder head of sales team france -> predicted: Account Executive | res: Business Developer
0.5268424153327942: développeur des solutions cybersécurité cybersecurity engineer intern cybersecurity intern security scholarship -> predicted: Consultant Cybersécurité | res: Ingénieur Cybersécurité
0.16805489361286

In [ ]:
model = create_model(0.0001, 0.3)
model.load_weights(weight_path)
weight_path = '{}_my_model_improved.best.hdf5'.format('xray_class')

#'weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min', save_weights_only= True)
early = EarlyStopping(monitor= 'val_loss', mode= 'min', patience= 100)
callbacks_list = [checkpoint, early]
cnn_model = model.fit(
    x=x_train,
    y=y_train,
    batch_size=32,
    validation_data=(x_val, y_val),
    epochs=5,
    callbacks= callbacks_list
)

NameError: name 'x_val' is not defined